In [ ]:
from os import listdir
from os.path import isfile, join
from skimage import io
from math import floor
from random import shuffle
import numpy as np
import cv2
import matplotlib.pyplot as plt
from itertools import chain
%matplotlib inline  
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

WINDOW_SIZE = 256
STRIDE = 256

def sliding_window(img, stride):
    h = img.shape[0]
    w = img.shape[1]
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            window = img[y:y+WINDOW_SIZE, x:x+WINDOW_SIZE]
            if window.shape[0] != WINDOW_SIZE or window.shape[1] != WINDOW_SIZE:
                continue
            yield window


def validation_set(path):
    batch_x = []
    batch_y = []
    img_files = [f for f in listdir(join(path, "img")) if isfile(join(path, "img", f))]
    for i, f in enumerate(img_files):
        img = cv2.imread(join(path, "img", f))
        label = cv2.imread(join(path, "label", f), 0)
        if label is None:
            print("{} not found".format(f))
            continue
        assert img.shape[0] == label.shape[0] and img.shape[1] == label.shape[1]
        img_windows = sliding_window(img,WINDOW_SIZE)
        label_windows = sliding_window(label, WINDOW_SIZE)
        samples = zip(img_windows, label_windows)
        for img_window, label_window in samples:
            if img_window.shape[0] != WINDOW_SIZE or img_window.shape[1] != WINDOW_SIZE:
                continue
            if label_window.shape[0] != WINDOW_SIZE or label_window.shape[1] != WINDOW_SIZE:
                continue
            batch_x.append(img_window)
            batch_y.append(label_window.reshape(WINDOW_SIZE,WINDOW_SIZE,1)/255)
    return np.array(batch_x), np.array(batch_y)

def load_imgs(path, f):
    img = cv2.imread(join(path, "img", f))
    label = cv2.imread(join(path, "label", f), 0)
    assert img.shape[0] == label.shape[0] and img.shape[1] == label.shape[1]
    return img, label


def make_transformations(img):
    result = []
    tmp = img
    result.append(tmp)
    result.append(np.fliplr(tmp))
    result.append(np.flipud(tmp))
    tmp = np.rot90(img,1)
    result.append(tmp)
    result.append(np.fliplr(tmp))
    result.append(np.flipud(tmp))
    tmp = np.rot90(img,2)
    result.append(tmp)
    result.append(np.fliplr(tmp))
    result.append(np.flipud(tmp))
    tmp = np.rot90(img,3)
    result.append(tmp)
    result.append(np.fliplr(tmp))
    result.append(np.flipud(tmp))
    return result

    
def prepare_samples(path,f, transformations = False):
    x_results = []
    y_results = []
    img, label = load_imgs(path, f)
    img_windows = iter([])
    label_windows = iter([])
    if transformations:
        t_imgs  = make_transformations(img)
        t_labels = make_transformations(label)
        for t_img, t_label in zip(t_imgs,t_labels):
            img_windows = chain(img_windows,sliding_window(img, STRIDE))
            label_windows = chain(label_windows, sliding_window(label, STRIDE))
    else:
        img_windows = chain(img_windows,sliding_window(img, STRIDE))
        label_windows = chain(label_windows, sliding_window(label, STRIDE))
    samples = zip(img_windows, label_windows)
    for img_window, label_window in samples:
        label_window = label_window.astype("float64")
        if(label_window.max() > 0):
            label_window = label_window / label_window.max()

        if np.count_nonzero(img_window == [255,255,255]) > WINDOW_SIZE**2 * 0.1:
            continue
        if np.count_nonzero(label_window == 1.0) < 100:
            continue
        if img_window.shape[0] != WINDOW_SIZE or img_window.shape[1] != WINDOW_SIZE:
            continue
        if label_window.shape[0] != WINDOW_SIZE or label_window.shape[1] != WINDOW_SIZE:
            continue
        x_results.append(img_window)
        y_results.append(label_window.reshape(WINDOW_SIZE,WINDOW_SIZE,1))
    return x_results, y_results
    
def generate_samples(path, batch_size, loop=True, transformations = False):
    img_files = [f for f in listdir(join(path, "img")) if isfile(join(path, "img", f))]
    batch_counter = 0
    batch_x = []
    batch_y = []
    while True:
        shuffle(img_files)
        for i, f in enumerate(img_files):
            a,b = prepare_samples(path, f, transformations = transformations)
            samples = zip(a,b)
            for img_window, label_window in samples:
                if batch_counter == batch_size:
                   # print(np.array(batch_x).shape, np.array(batch_y).shape)
                    yield np.array(batch_x), np.array(batch_y)
                    batch_counter = 0
                    batch_x = []
                    batch_y = []
                batch_x.append(img_window)
                batch_y.append(label_window)
                batch_counter +=1
        if not loop:
            break


#for sample in generate_samples("/home/sirpawel/piro_3/data/train_batch_1_raw", 32):
    #pass


In [ ]:


def save_data_set(imgs,labels, path, idx):
    for img, label in zip(imgs,labels):
        cv2.imwrite(join(path, "img/", str(idx) + ".tiff" ),img)
        cv2.imwrite(join(path, "label/", str(idx) + ".tiff" ),label)
        
idx = 0;
for sample in generate_samples("/home/sirpawel/piro_3/data/sample_raw", 1, loop=False):
    save_data_set(sample[0],sample[1], "/home/sirpawel/piro_3/data/sample", idx)
    idx +=1
print(idx)

In [ ]:
def count_samples(path):
    counter = 0
    img_files = [f for f in listdir(join(path, "img")) if isfile(join(path, "img", f))]
    for i, f in enumerate(img_files):
        #print("file {}/{}".format(i+1,len(img_files)))
        a,b = prepare_samples(path, f, transformations=True)
        counter += len(a)
    return counter
print(count_samples("/home/sirpawel/piro_3/data/train_raw")) 

In [ ]:
def join_with_labels(img, label):
        label = label.squeeze()
        background = np.full((img.shape[0],img.shape[1],3), (255,0,255)).astype("uint8")
        label_mask = np.zeros((img.shape[0], img.shape[1])).astype("uint8")
        label_mask[0+label.shape[0]//2:img.shape[0]-label.shape[0]//2,0+label.shape[1]//2:img.shape[1]-label.shape[1]//2] = label
        m1 = cv2.bitwise_and(background, background, mask = label_mask)
        masked = cv2.bitwise_or(img, m1)
        return masked
        
a, b = prepare_samples("/home/sirpawel/piro_3/data/train_batch_1_raw/","10078660_15.tiff")

counter = 0



In [ ]:
import keras
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization
from keras import backend as K
from keras.models import Model



def down(input_layer, filters, pool=True):
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(input_layer)
    conv1_batch = BatchNormalization()(conv1)
    residual = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1_batch)
    res_batch = BatchNormalization()(residual)
    if pool:
        max_pool = MaxPooling2D((2,2))(res_batch)
        return max_pool, res_batch
    else:
        return res_batch

def up(input_layer, residual, filters):
    filters=int(filters)
    upsample = UpSampling2D()(input_layer)
    upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
    upconv_batch = BatchNormalization()(upconv)
    
    concat = Concatenate(axis=3)([residual, upconv_batch])
    
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(concat)
    conv1_batch = BatchNormalization()(conv1)
    
    conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1_batch)
    conv2_batch = BatchNormalization()(conv2)
    return conv2_batch

filters = 16
input_layer = Input(shape = [WINDOW_SIZE, WINDOW_SIZE, 3])

#256->128
d1, res1 = down(input_layer, filters)

filters *= 2

#128->64
d2, res2 = down(d1, filters)

filters *= 2

#64->32
d3, res3 = down(d2, filters)

filters *= 2

#32->16
d4, res4 = down(d3, filters)

dense1 = Dense(128, activation='relu')(d4)
dense1_batch = BatchNormalization()(dense1)
drop = Dropout(0.5)(dense1_batch)
dense2 = Dense(128, activation='relu')(drop)
dense2_batch = BatchNormalization()(dense2)


#16->32
up4 = up(dense2_batch, res4, filters)

filters /= 2

#32->64
up3 = up(up4, res3, filters)

filters /= 2

#64->128
up2 = up(up3, res2, filters)

filters /= 2

up1 = up(up2, res1, filters)


out = Conv2D(1, (3, 3), padding='same', activation='sigmoid')(up1)


model = Model(input_layer, out)


import keras.backend as K

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
modelCheckpoint = keras.callbacks.ModelCheckpoint('model_best.hd5', monitor='val_loss', save_best_only=True,)



model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy', recall, precision],
             callbacks = [earlyStopping,modelCheckpoint])

model.summary()




In [ ]:


validation_x, validation_y = validation_set("/home/sirpawel/piro_3/data/val_raw")
print(validation_x.shape)
print(validation_y.shape)
#x = []
#y = []
#for sample in generate_samples("/home/sirpawel/piro_3/data/train_batch_1_raw", 1, loop=False):
#    x.append(sample[0][0])
#    y.append(sample[1][0])
    #plt.figure()
    #io.imshow(sample[0][0])
    #plt.figure()
    #io.imshow(sample[1][0].reshape((256,256)))
    
#x = np.array(x)
#y = np.array(y)



#print(x.shape)
#print(y.shape)
#536484
from keras.models import load_model
model = load_model('model.hd5',custom_objects={'recall': recall, 'precision': precision})
model.fit_generator(
    generate_samples("/home/sirpawel/piro_3/data/train_raw", 16, transformations = True),
    epochs = 3, 
    steps_per_epoch=134121/16,
    validation_data = (validation_x, validation_y))


#model.fit(x = x, y = y, batch_size = 32, epochs = 100, validation_data = (validation_x, validation_y))
model.save("model2.hd5")

In [ ]:
from keras.models import load_model
model = load_model('model2.hd5',custom_objects={'recall': recall, 'precision': precision})



In [2]:
import keras.backend as K
import cv2
import numpy as np
from keras.models import load_model


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

WINDOW_SIZE = 256
def sliding_window(img):
    h = img.shape[0]
    w = img.shape[1]
    if h % WINDOW_SIZE != 0 or w % WINDOW_SIZE != 0:
        h_pad = WINDOW_SIZE - (h % WINDOW_SIZE)
        w_pad = WINDOW_SIZE - (w % WINDOW_SIZE)
        img = cv2.copyMakeBorder(img, 0, h_pad, 0, w_pad, cv2.BORDER_CONSTANT, None, (0,0,0))
    for y in range(0, h, WINDOW_SIZE):
        for x in range(0, w, WINDOW_SIZE):
            window = img[y:y+WINDOW_SIZE, x:x+WINDOW_SIZE]
            if window.shape[0] != WINDOW_SIZE or window.shape[1] != WINDOW_SIZE:
                continue
            yield window
            
def reconstruct(imgs, h,w):
    mask = np.zeros((h + (WINDOW_SIZE - (h % WINDOW_SIZE)), w + ( WINDOW_SIZE - (w % WINDOW_SIZE))))
    idx = 0
    for row in range(0, mask.shape[0] // WINDOW_SIZE):
        for col in range(0, mask.shape[1] // WINDOW_SIZE):
            mask[row * WINDOW_SIZE : (row+1)*WINDOW_SIZE, col * WINDOW_SIZE : (col+1)*WINDOW_SIZE] = imgs[idx].squeeze()
            idx +=1
    return np.round(mask[0:h,0:w])

def roads(image):
    model = load_model('model2.hd5',custom_objects={'recall': recall, 'precision': precision})
    imgs = np.array(list(sliding_window(image)))
    results = model.predict(np.array(imgs))
    return reconstruct(results, image.shape[0], image.shape[1])



img = cv2.imread("/home/sirpawel/piro_3/data/val_raw/img/10228690_15.tiff")
result = roads(img)




In [ ]:
a = [[1, 2], [3, 4]]
np.pad(a, ((0, 36), (0, 36)), 'constant', constant_values=(0, 0))


In [ ]:
def join_with_labels(img, label1, label2):
        label1 = label1.squeeze()
        label2 = label2.squeeze()
        background = np.full((img.shape[0],img.shape[1],3), (0,255,0)).astype("uint8")
        m1 = cv2.bitwise_and(background, background, mask = label1.astype("uint8"))
        masked = cv2.bitwise_or(img, m1)
        
        background = np.full((img.shape[0],img.shape[1],3), (255,0,0)).astype("uint8")
        m1 = cv2.bitwise_and(background, background, mask = label2.astype("uint8"))
        masked = cv2.bitwise_or(masked, m1)
        return masked

img = cv2.imread("/home/sirpawel/piro_3/data/val_raw/img/10228690_15.tiff")
label = cv2.imread("/home/sirpawel/piro_3/data/val_raw/label/10228690_15.tiff", 0)

result = roads(img)
labeled = join_with_labels(img,label,result)
io.imshow(labeled)
io.imsave("tmp/labeled.tiff", labeled)
